<a id='top'></a>
# Settling Down in Toronto: Applied Capstone Project

### Table of Contents
1. [Introduction](#intro)
2. [Data](#data)
3. [Methodology](#method)
4. [Results and Discussion](#results)
5. [Conclusion](#conclusion)

<a id = "intro"></a>
## Introduction

While I'm not at all planning on starting a family anytime soon, I have always been curious about how my parents scouted a good location for them to settle down and raise children. I think I'm more interested because, at the time, tech and techniques like those we've been taught so far in this program were not as accessible to my parents like they are now. 

It's easier now for us to do our research remotely—and now that we're in a pandemic, it's also safer. So, the main objective of this project is to explore neighbourhoods that might be of interest to anyone wanting to settle down and start a family. 

#### Problem
Considering that Toronto has 100+ neighbourhoods, how can we narrow them down to the most viable options for places to settle down and start a family?

#### We'll be exploring Toronto, in particular, and for 2 reasons:  
1. Canada is a country that I am genuinely interested in visiting as I've read and heard that quality of life is *relatively* better there than in a lot of places. So, it sounds like a good place to start exploring if you're planning to start a family soon and you have the means to go abroad (or you're already in Canada).
2. I've already got a dataset for it, which is very convenient and saves some time (admittedly, although I've got tons and tons of ideas for this project, I'm on a time crunch).


#### Who might be interested in this? 
Great question! Thanks for asking. You might be interested if:  
* you're interested in moving to Toronto. Depending on what you're looking for, this could help with your research.  
  
  
* you're looking to settle down and start a family soon. This provides a framework that you could apply to your own research—or even give you some ideas about what you might want to consider in your research.   
  
  
* you're also just starting out in your data science journey and want to work on a fun project similar to this. The techniques used in this notebook are among fundamentals that we've just been taught in this program, so you might find this useful.  
  
  
* you're my parents, because you want to know that this program was worth it (huge thank you, if you're reading this, and also just want to clarify that no, I'm not planning to run off to Canada and start a family just yet—I might just get a dog but that's about it until I'm even remotely financially stable).

<a id = "data"></a>
## Data

#### For this project, we'll be using the following data:

* [This](https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M) Wikipedia page with a list of neighbourhoods in Canada and their postal codes.
* [This](http://cocl.us/Geospatial_data) csv file of coordinates of different neighbourhoods in Canada. The file was provided by our instructors in the IBM Data Science Professional Certificate program.
* Foursquare Places API data

The original plan was to use the geocoder library to find the coordinates, but when I used it previously in the Segmenting and Clustering Neighborhoods in Toronto activity, it took a long time. Of course, that could be because of the script I used, but for now, we'll use the csv file for efficiency. 

As for the Foursquare API data, we'll be using this to get more insight into the venues (like shops, parks, etc.) that are within these neighbourhoods.

#### How we'll utilize this data:
We're going to use the Wikipedia page to get a list of the neighbourhoods assigned to each postal code. Once we have this, we can match the neighbourhoods to their coordinates using the given csv file. After we match them, we can then use the Foursquare Places API to get a breakdown of the venues (specific, e.g. Tim Horton) and the venue categories (general, e.g. Cafe).

Once we have this breakdown, we can then proceed to exploring the data further in terms of certain minimum requirements we'll set (i.e. neighbourhood must have a hospital).

#### Exploring the Wikipedia page on neighbourhoods in Canada

In order for us to explore this data more conveniently, we're going to have to perform a bit of web scraping so that we can turn the list table into a dataframe that we can manipulate with the pandas library.

In [1]:
import pandas as pd # library for data analysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import numpy as np
import os
import requests # library to handle requests
from bs4 import BeautifulSoup # library to parse HTML documents

print("Libraries imported.")

Libraries imported.


Awesome! As a heads up, I might link certain articles throughout this notebook as a guide for anyone going through this and for myself, as reference.

Now, we can start with the web scraping. For reference, I followed the guide here: https://medium.com/analytics-vidhya/web-scraping-a-wikipedia-table-into-a-dataframe-c52617e1f451.
According to the article, we must first check if it's legal for us to scrape a particular webpage. If we get a status code of 200, then it is. So let's hope we get 200.

In [2]:
wikiurl="https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
table_class="wikitable sortable jquery-tablesorter"
response=requests.get(wikiurl)

if response.status_code == 200:
    print(response.status_code, "Commence the scraping!")
else:
    print(response.status_code)

200 Commence the scraping!


We're going to read the table (which has the list of postal codes, boroughs, and neighbourhoods) into a dataframe. 

In [3]:
soup = BeautifulSoup(response.text, 'html.parser')
pctable=soup.find('table',{'class':"wikitable"})

df=pd.read_html(str(pctable))
# convert list to dataframe
df=pd.DataFrame(df[0])
df.head()

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


As we can see from the dataframe above, we now have the following information:

* Postal Codes
* Boroughs
* Neighbourhoods

Later, we'll see how we can find more details on the venues within these neighbourhoods.

Note, however, that there are some boroughs and neighbourhoods that are unassigned. Since we only want to navigate those that *are* assigned, let's filter them out.

In [4]:
index_names = df[ df['Borough'] == "Not assigned" ].index 
df.drop(index_names, inplace = True)
df.index = range(len(df))
print("Shape:", df.shape)
df.head()

Shape: (103, 3)


,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


#### Getting the coordinates for each neighbourhood

Now that we have the list of boroughs and neighbourhoods, we're going to take the csv file with coordinates and create a new dataframe with all of them combined.

In [5]:
coord_df = pd.read_csv("http://cocl.us/Geospatial_data")
coord_df.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


We notice that the order of the postal codes in this dataframe is different than the order in the first one. So, we'll have to consider this when we match locations to their coordinates.

The way that we'll do this is zip the three columns of coord_df so we can now work with a list of tuples. Then, we'll create do the same for df.

In [6]:
coords = list(zip(coord_df["Postal Code"], coord_df["Latitude"], coord_df["Longitude"]))
coord_pc = list(coord_df["Postal Code"])
print("Coordinates:\n", coords[:5])

areas = list(zip(df["Postal Code"], df["Borough"], df["Neighbourhood"]))
print("\nAreas:\n", areas[:5])

#Just checking:
print("\nCoords Length:", len(coords))
print("\nAreas Length:", len(areas))

Coordinates:
 [('M1B', 43.806686299999996, -79.19435340000001), ('M1C', 43.7845351, -79.16049709999999), ('M1E', 43.7635726, -79.1887115), ('M1G', 43.7709921, -79.21691740000001), ('M1H', 43.773136, -79.23947609999999)]

Areas:
 [('M3A', 'North York', 'Parkwoods'), ('M4A', 'North York', 'Victoria Village'), ('M5A', 'Downtown Toronto', 'Regent Park, Harbourfront'), ('M6A', 'North York', 'Lawrence Manor, Lawrence Heights'), ('M7A', 'Downtown Toronto', "Queen's Park, Ontario Provincial Government")]

Coords Length: 103

Areas Length: 103


Now that we have the lists, we'll do what can be pretty time-consuming for longer lists. We'll create a list of dictionaries from the zipped lists to create a new dataframe with correctly matched postal codes from df and coordinates from coord_df.

In [7]:
def new_df(areas, coords, df_out_name):
    rows = []
    for area in areas:
        df_out_dict = {}
        
        if area[0] in coord_pc:
            index = coord_pc.index(area[0])
        
            d_new = {"Postal Code": area[0], "Borough": area[1], "Neighbourhood": area[2], "Latitude": coords[index][1], "Longitude": coords[index][2]}
        
            df_out_dict.update(d_new)
        
            rows.append(df_out_dict)
        
    df_toronto = pd.DataFrame(rows, columns = ["Postal Code", "Borough", "Neighbourhood", "Latitude", "Longitude"])
    df_toronto.to_csv(df_out_name, index=False)
    return df_toronto.head()
    
new_df(areas, coords, "df_toronto.csv")

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


And it works! Now we have a dataframe with neighbourhoods and their coordinates. 

In a little while, we're going to start clustering these neighbourhoods based on some criteria to help us narrow our options for places to settle down. As another heads up, we'll be using the Foursquare Places API to get the info we need on the venues in these areas.

#### Exploring Toronto

In [8]:
#This is to get an idea of how many neighbourhoods we've got initially.
df_toronto = pd.read_csv("df_toronto.csv")

print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(df_toronto['Borough'].unique()),
        df_toronto.shape[0]
    )
)

The dataframe has 10 boroughs and 103 neighborhoods.


In [9]:
#Just importing some more libraries we'll need
from geopy.geocoders import Nominatim 
from pandas.io.json import json_normalize

import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans

import folium 

print('Libraries imported.')

Libraries imported.


In [10]:
#We're going to set our user_agent to "ca_explorer" for fun. But you can use a different string.

address = 'Downtown Toronto, CA'

geolocator = Nominatim(user_agent="ca_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinates of Downtown Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinates of Downtown Toronto are 43.6563221, -79.3809161.


One thing I must clarify here is that, previously, I'd set the address to 'Toronto, CA'; however, the markers did not appear on the map that we'll generate next. So, feel free to try different addresses. This is just what worked for me. 

Once we generate the map, you'll see the markers on different spots on the map. Upon clicking them, you'll see the label of that location.

In [11]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for latitude, lng, borough, neighbourhood in zip(df_toronto['Latitude'], df_toronto['Longitude'], df_toronto['Borough'], df_toronto['Neighbourhood']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [latitude, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

Recall from the dataframe that there are boroughs without the word "Toronto" in their labels. Here's what you can do to only set markers for those with "Toronto".

In [12]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for latitude, lng, borough, neighbourhood in zip(df_toronto['Latitude'], df_toronto['Longitude'], df_toronto['Borough'], df_toronto['Neighbourhood']):
    if "Toronto" in borough: #if we only want to see boroughs with "Toronto" in their names
        label = '{}, {}'.format(neighbourhood, borough)
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [latitude, lng],
            radius=5,
            popup=label,
            color='blue',
            fill=True,
            fill_color='#3186cc',
            fill_opacity=0.7,
            parse_html=False).add_to(map_toronto)  
    
map_toronto

In [13]:
#dummy app credentials
CLIENT_ID = os.environ.get('Foursquare_User') # your Foursquare ID
CLIENT_SECRET = os.environ.get('Foursquare_Secret') # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: *****')
print('CLIENT_SECRET: *****')

Your credentails:
CLIENT_ID: *****
CLIENT_SECRET: *****


And now we're going to use the Foursquare Places API to help us find out what venues (like shops) are accessible in the neighbourhoods we've got listed. Remember, since we're looking for viable options for places to settle down and start a family, there are certain venues we want nearby.

At minimum, we need a **School** and a **Hospital** present closeby. But, we'd also love to have a **Park** nearby, as well. Let's see!

In [14]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [18]:
toronto_venues = getNearbyVenues(names=df_toronto['Neighbourhood'],
                                   latitudes=df_toronto['Latitude'],
                                   longitudes=df_toronto['Longitude']
                                  )

Parkwoods
Victoria Village
Regent Park, Harbourfront
Lawrence Manor, Lawrence Heights
Queen's Park, Ontario Provincial Government
Islington Avenue, Humber Valley Village
Malvern, Rouge
Don Mills
Parkview Hill, Woodbine Gardens
Garden District, Ryerson
Glencairn
West Deane Park, Princess Gardens, Martin Grove, Islington, Cloverdale
Rouge Hill, Port Union, Highland Creek
Don Mills
Woodbine Heights
St. James Town
Humewood-Cedarvale
Eringate, Bloordale Gardens, Old Burnhamthorpe, Markland Wood
Guildwood, Morningside, West Hill
The Beaches
Berczy Park
Caledonia-Fairbanks
Woburn
Leaside
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Bathurst Manor, Wilson Heights, Downsview North
Thorncliffe Park
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Scarborough Village
Fairview, Henry Farm, Oriole
Northwood Park, York University
East Toronto, Broadview North (Old East York)
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
Kennedy Park, Ionview, East Birchmo

In [19]:
print(toronto_venues.shape)
toronto_venues.head()

(2141, 7)


,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
1,Parkwoods,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop
2,Victoria Village,43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena
3,Victoria Village,43.725882,-79.315572,Tim Hortons,43.725517,-79.313103,Coffee Shop
4,Victoria Village,43.725882,-79.315572,Portugril,43.725819,-79.312785,Portuguese Restaurant


<a id='method'></a>
## Methodology

Awesome! Now, we can start to navigate the venues.

#### How are we going to navigate them?

Good question. This is the plan. First, we'll group the data by neighbourhood. It's really cool that we can also see the venue categories, because this adds a level of uniformity in the keywords that we'll use to see which neighbourhoods meet our minimum.

As a refresher (with additions), the following venue categories in no particular order are those we'd like to have in the neighbourhood at minimum:
* Hospital or Clinic
* School
* Drugstore
* Park
* Farmers Market or Grocery Store

Note: For the purpose of this project, we're going to assume that housing and source of income are either already taken care of or are simply not variables we must factor in. 

After we shortlist neighbourhoods, we can then look into clustering them based on the most common venues (i.e. which venue categories have the highest count). This way, we can also get a feel of the sort of people who might be your neighbours (if there are many of these venues, then we consider that the target market would reside or often visit the neighbourhood) *and* find out what venues of interest other than our minimums might be accessible.

In [20]:
toronto_venues.groupby('Neighbourhood').count()

,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,
Agincourt,5,5,5,5,5,5
"Alderwood, Long Branch",7,7,7,7,7,7
"Bathurst Manor, Wilson Heights, Downsview North",21,21,21,21,21,21
Bayview Village,4,4,4,4,4,4
"Bedford Park, Lawrence Manor East",22,22,22,22,22,22
Berczy Park,55,55,55,55,55,55
"Birch Cliff, Cliffside West",4,4,4,4,4,4
"Brockton, Parkdale Village, Exhibition Place",23,23,23,23,23,23
"Business reply mail Processing Centre, South Central Letter Processing Plant Toronto",16,16,16,16,16,16


In [21]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 273 uniques categories.


In [22]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighbourhood'] = toronto_venues['Neighbourhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Neighbourhood,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Business Service,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Cafeteria,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Escape Room,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indoor Play Area,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kids Store,Kitchen Supply Store,Korean BBQ Restaurant,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Locksmith,Lounge,Luggage Store,Malay Restaurant,Market,Martial Arts School,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Motel,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Print Shop,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,River,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Social Club,Soup Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,Parkwoods,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [23]:
toronto_onehot.shape

(2141, 274)

#### Shortlisting Neighbourhoods
In order to shortlist neighbourhoods, we need to see if they meet our minimum requirements, as listed previously. In order to do this, we'll first generate a dataframe that tells us how many of each type of venue is accessible within each neighbourhood. 

In [24]:
toronto_count = toronto_onehot.groupby('Neighbourhood').sum().reset_index()
toronto_count.head()

,Neighbourhood,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Business Service,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Cafeteria,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Escape Room,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indoor Play Area,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kids Store,Kitchen Supply Store,Korean BBQ Restaurant,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Locksmith,Lounge,Luggage Store,Malay Restaurant,Market,Martial Arts School,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Motel,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Print Shop,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,River,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Social Club,Soup Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,Agincourt,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

Now that we've got a dataframe that tells us how many venues of each venue category are present in each neighbourhood, we can shortlist the neighbourhoods that meet our requirements.

In order to do this, we'll convert this dataframe into a list which we'll then iterate over.

In [25]:
#we need to get the position of the venue categories that satisfy our requirements
reqs = ['Hospital', 'Clinic', 'School', 'Pharmacy', 'Drugstore', 'Park', 'Farmers Market', 'Grocery']
toronto_dict = toronto_count.to_dict()

#each key is a column label and each sub-dictionary is a separate column

def find_index(df):
    key_index = []
    for key in df.keys():
        for req in reqs:
            if req in key:
                index = list(df.keys()).index(key)
                key_index.append([key,index])
    return key_index
                
key_index = find_index(toronto_dict)
key_index

[['Drugstore', 92],
 ['Farmers Market', 100],
 ['Grocery Store', 130],
 ['Hospital', 144],
 ['Martial Arts School', 174],
 ['Organic Grocery', 199],
 ['Park', 201],
 ['Pharmacy', 204],
 ['Skate Park', 232],
 ['Swim School', 252]]

Okay, so we see here that, actually, this list of venue categories does not include elementary or highschools. But since we have all of our other requirements satisfied, let's now assume that schooling is taken care of (basically, we'll remove the two schools from our key_index list) and proceed with our exploration of the neighbourhoods.

In [26]:
key_index.remove(['Martial Arts School', 174])
key_index.remove(['Swim School', 252])

In [27]:
toronto_list = toronto_count.values.tolist()
rows = len(toronto_list)
#print(toronto_list[:5])
shortlist = []

def check(row, reqs, key_index, row_index):
    req_count = 0 #to count how many of our minimum requirements are satisfied
    for i in key_index: 
        for req in reqs:
            if (req in i[0]) and (row[i[1]] > 0):
                req_count = req_count + 1
    if req_count >= 3: 
    #We're setting this to 3 because I tried with at least 5 and only 1 neighbourhood popped up.
    #But since some of the elements in key_index are of the same type (i.e. park vs skate park),
    #we'll be considering all neighbourhoods that can satisfy at least 3 of our requirements.
        return True
    else: 
        return False
    

for row_index in range(rows):
    row = toronto_list[row_index]
    check_req = check(row, reqs, key_index, row_index)
    if check_req:
        shortlist.append(row)
print(f"We have shortlisted {len(shortlist)} neighbourhoods!\n")

toronto_shortlist = pd.DataFrame(shortlist, columns = toronto_dict.keys())

We have shortlisted 9 neighbourhoods!



Now that we have our shortlist, we'll cluster them and generate a map to (1) see any commonolaties for reference when taking a decision and (2) visualize them better.

In [28]:
toronto_grouped = toronto_onehot.groupby('Neighbourhood').mean().reset_index()
#toronto_grouped.head()
#We need to get only the shortlisted ones.
neighs = []
for _ in shortlist:
    neighs.append(_[0])
#print(neighs)

toronto_grouped.set_index('Neighbourhood', inplace=True)
toronto_shortlist_grouped = toronto_grouped.loc[neighs]
toronto_shortlist_grouped.reset_index(inplace=True)
toronto_shortlist_grouped

,Neighbourhood,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Business Service,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Cafeteria,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Escape Room,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indoor Play Area,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kids Store,Kitchen Supply Store,Korean BBQ Restaurant,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Locksmith,Lounge,Luggage Store,Malay Restaurant,Market,Martial Arts School,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Motel,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Print Shop,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,River,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Social Club,Soup Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,Berczy Park,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0

In [29]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [30]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighbourhood'] = toronto_shortlist_grouped['Neighbourhood']

for ind in np.arange(toronto_shortlist_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_shortlist_grouped.iloc[ind, :], num_top_venues)

#### Clustering Neighbourhoods
For this, we'll be doing k-means clustering.

In [31]:
# set number of clusters
kclusters = 5

toronto_shortlist_grouped_clustering = toronto_shortlist_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_shortlist_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 3, 4, 2, 0, 0, 0, 0, 1])

In [32]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = df_toronto

# merge to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')

toronto_merged.dropna(inplace=True)

toronto_merged.reset_index(inplace=True) # check the last columns!

In [33]:
toronto_merged.drop('index', axis=1 ,inplace=True)
toronto_merged

,Postal Code,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,0.0,Coffee Shop,Café,Restaurant,Cocktail Bar,Beer Bar,Gastropub,American Restaurant,Seafood Restaurant,Farmers Market,Hotel
1,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306,0.0,Coffee Shop,Restaurant,Cheese Shop,Bakery,Cocktail Bar,Beer Bar,Farmers Market,Seafood Restaurant,Sandwich Place,Breakfast Spot
2,M4H,East York,Thorncliffe Park,43.705369,-79.349372,1.0,Sandwich Place,Indian Restaurant,Yoga Studio,Grocery Store,Burger Joint,Bus Line,Restaurant,Coffee Shop,Pizza Place,Pharmacy
3,M6H,West Toronto,"Dufferin, Dovercourt Village",43.669005,-79.442259,2.0,Pharmacy,Bakery,Music Venue,Middle Eastern Restaurant,Café,Brewery,Bar,Supermarket,Bank,Park
4,M4S,Central Toronto,Davisville,43.704324,-79.388790,4.0,Sandwich Place,Dessert Shop,Pizza Place,Gym,Coffee Shop,Café,Italian Restaurant,Sushi Restaurant,Brewery,Greek Restaurant
5,M5T,Downtown Toronto,"Kensington Market, Chinatown, Grange Park",43.653206,-79.400049,0.0,Café,Mexican Restaurant,Vegetarian / Vegan Restaurant,Coffee Shop,Bar,Vietnamese Restaurant,Dessert Shop,Dumpling Restaurant,Burger Joint,Bakery
6,M5W,Downtown Toronto,Stn A PO Boxes,43.646435,-79.374846,0.0,Coffee Shop,Italian Restaurant,Restaurant,Pub,Seafood Restaurant,Café,Beer Bar,Hotel,Japanese Restaurant,Breakfast Spot
7,M4X,Downtown Toronto,"St. James Town, Cabbagetown",43.667967,-79.367675,0.0,Coffee Shop,Restaurant,Café,Pizza Place,Chinese Restaurant,Italian Restaurant,Market,Pub,Bakery,Park
8,M7Y,East Toronto,"Business reply mail Processing Centre, South C...",43.662744,-79.321558,3.0,Gym / Fitness Center,Garden Center,Farmers Market,Fast Food Restaurant,Light Rail Station,Burrito Place,Butcher,Restaurant,Recording Studio,Brewery


In [46]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighbourhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster-1)],
        fill=True,
        fill_color=rainbow[int(cluster-1)],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

<a id='results'></a>
## Results and Discussion

In this section, we'll discuss the neighbourhoods we've shortlisted, as well as their clusters and how this research can be improved/developed further.

In [47]:
toronto_shortlist

,Neighbourhood,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Business Service,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Cafeteria,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Escape Room,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indoor Play Area,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kids Store,Kitchen Supply Store,Korean BBQ Restaurant,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Locksmith,Lounge,Luggage Store,Malay Restaurant,Market,Martial Arts School,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Motel,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Print Shop,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,River,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Social Club,Soup Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,Berczy Park,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,2,0,0,0,0

One thing we can see straight away is that none of these shortlisted neighbourhoods have an airport. So, assuming you're not planning on traveling frequently, this is would be great for settling down if we consider how much noise you'd hear from an airport.

Since we've already shortlisted these neighbourhoods based on our requirements, let's see the top 10 most common venues in each so that we can get a feel of their profiles.

In [48]:
neighborhoods_venues_sorted

,Cluster Labels,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,0,Berczy Park,Coffee Shop,Restaurant,Cheese Shop,Bakery,Cocktail Bar,Beer Bar,Farmers Market,Seafood Restaurant,Sandwich Place,Breakfast Spot
1,3,"Business reply mail Processing Centre, South C...",Gym / Fitness Center,Garden Center,Farmers Market,Fast Food Restaurant,Light Rail Station,Burrito Place,Butcher,Restaurant,Recording Studio,Brewery
2,4,Davisville,Sandwich Place,Dessert Shop,Pizza Place,Gym,Coffee Shop,Café,Italian Restaurant,Sushi Restaurant,Brewery,Greek Restaurant
3,2,"Dufferin, Dovercourt Village",Pharmacy,Bakery,Music Venue,Middle Eastern Restaurant,Café,Brewery,Bar,Supermarket,Bank,Park
4,0,"Kensington Market, Chinatown, Grange Park",Café,Mexican Restaurant,Vegetarian / Vegan Restaurant,Coffee Shop,Bar,Vietnamese Restaurant,Dessert Shop,Dumpling Restaurant,Burger Joint,Bakery
5,0,St. James Town,Coffee Shop,Café,Restaurant,Cocktail Bar,Beer Bar,Gastropub,American Restaurant,Seafood Restaurant,Farmers Market,Hotel
6,0,"St. James Town, Cabbagetown",Coffee Shop,Restaurant,Café,Pizza Place,Chinese Restaurant,Italian Restaurant,Market,Pub,Bakery,Park
7,0,Stn A PO Boxes,Coffee Shop,Italian Restaurant,Restaurant,Pub,Seafood Restaurant,Café,Beer Bar,Hotel,Japanese Restaurant,Breakfast Spot
8,1,Thorncliffe Park,Sandwich Place,Indian Restaurant,Yoga Studio,Grocery Store,Burger Joint,Bus Line,Restaurant,Coffee Shop,Pizza Place,Pharmacy


Just from looking at this dataframe, we can see that majority of the most common venues are places where people go to eat. But now we want to explore the clusters.

#### Cluster 1

In [49]:
cluster_1 = toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]
cluster_1.reset_index(inplace=True)
cluster_1.drop('index', axis=1, inplace=True)
cluster_1

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Downtown Toronto,0.0,Coffee Shop,Café,Restaurant,Cocktail Bar,Beer Bar,Gastropub,American Restaurant,Seafood Restaurant,Farmers Market,Hotel
1,Downtown Toronto,0.0,Coffee Shop,Restaurant,Cheese Shop,Bakery,Cocktail Bar,Beer Bar,Farmers Market,Seafood Restaurant,Sandwich Place,Breakfast Spot
2,Downtown Toronto,0.0,Café,Mexican Restaurant,Vegetarian / Vegan Restaurant,Coffee Shop,Bar,Vietnamese Restaurant,Dessert Shop,Dumpling Restaurant,Burger Joint,Bakery
3,Downtown Toronto,0.0,Coffee Shop,Italian Restaurant,Restaurant,Pub,Seafood Restaurant,Café,Beer Bar,Hotel,Japanese Restaurant,Breakfast Spot
4,Downtown Toronto,0.0,Coffee Shop,Restaurant,Café,Pizza Place,Chinese Restaurant,Italian Restaurant,Market,Pub,Bakery,Park


We see here that all 5 are in Downtown Toronto and that, for all of them, their 1st most common venue is some sort of coffee shop.

Considering that majority of the most common venues here are eateries and there are several bars and pubs, as well, it's possible that people who reside here are relatively young and quite busy. We're also making this assumption based on the fact that nowhere in these top 10 is a pharmacy, drugstore, or hospital.

#### Cluster 2

In [76]:
cluster_2 = toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]
cluster_2.reset_index(inplace=True)
cluster_2.drop('index', axis=1, inplace=True)
cluster_2

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,East York,1.0,Sandwich Place,Indian Restaurant,Yoga Studio,Grocery Store,Burger Joint,Bus Line,Restaurant,Coffee Shop,Pizza Place,Pharmacy


Since there were only 9 neighbourhoods on our shortlist, it makes sense that there would be some clusters with only 1 neighbourhood.

With the bus line being 6th most common venue, it's possible people here tend to travel quite a bit. This group of venues I find interesting, because it's a mix of places like the Pizza Place and those like the Yoga Studio. 

I'm not quite sure what to make of it with such limited data, but considering the Yoga Studio comes in as 3rd most common venue, I think it's rational to posit that people living here are relatively health-conscious.

#### Cluster 3

In [77]:
cluster_3 = toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]
cluster_3.reset_index(inplace=True)
cluster_3.drop('index', axis=1, inplace=True)
cluster_3

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,West Toronto,2.0,Pharmacy,Bakery,Music Venue,Middle Eastern Restaurant,Café,Brewery,Bar,Supermarket,Bank,Park


Interestingly, the most common venue here is the pharmacy. It's possible that there are a lot of elderly people in this neighbourhood, but it's also possible that there are many pharmacies for other reasons (e.g. clinic or hospital in a nearby neighbourhood that didn't make it to our shortlist).

#### Cluster 4

In [50]:
cluster_4 = toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]
cluster_4.reset_index(inplace=True)
cluster_4.drop('index', axis=1, inplace=True)
cluster_4

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,East Toronto,3.0,Gym / Fitness Center,Garden Center,Farmers Market,Fast Food Restaurant,Light Rail Station,Burrito Place,Butcher,Restaurant,Recording Studio,Brewery


So, this has a gym or fitness center as *the* most common venue and the Farmers Market as the 3rd most common venue, so perhaps the people living here are quite health-conscious too, just like in the Sandwich Cluster. I say "perhaps" because the 4th most common venue is Fast Food Restaurant. 

Another thing to note here is that the 5th most common venue is the Light Rail Station. It might be a bit noisy here because of that. 

#### Cluster 5

In [51]:
cluster_5 = toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]
cluster_5.reset_index(inplace=True)
cluster_5.drop('index', axis=1, inplace=True)
cluster_5

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Central Toronto,4.0,Sandwich Place,Dessert Shop,Pizza Place,Gym,Coffee Shop,Café,Italian Restaurant,Sushi Restaurant,Brewery,Greek Restaurant


This cluster, just like Cluster 1, has a lot of eateries for common venues. I'm wondering if the cost of going out to eat might be a bit less here than at the other clusters, simply because of the competition. But, perhaps not. For all we know, the restaurants are a chain and there isn't much competition because they're all different cuisines. 

Either way, if you can afford it, you've got a *lot* of options for dining. 

### Recommendations and Some Notes

It has become extremely apparent that we've been working with a very limited set of data. What I mean is that, the assumptions we've been making have been based purely on how many of different venue categories are present in each of our clusters. 

The problem with this is that we didn't really get a 360-view of these neighbourhoods. Here is a list of some of the information that anyone wanting to further this might want to consider:
* data on elementary schools, high schools, and daycare centers
* data on housing options in these neighbourhoods
* cost of living in these clusters or even in the boroughs
* job options
* population density
* crime rate

Such information would definitely help in getting better insight into these neighbourhoods, especially in terms of shortlisting which would be most conducive for settling down and raising and family.

<a id='conclusion'></a>
## Conclusion
The main objective of this project was to narrow down the neighbourhoods of Toronto and figure out which ones might be viable options for those looking to raise a family. 

We did some web scraping to get the list of neighbourhoods and matched them to their coordinates from a csv file provided by the course instructors. We then utilized the FourSquare Places API in order to get a summary of sorts of the venues within these places.

Next, we shortlisted some neighbourhoods based on minimum requirements, which are the following in no particular order:
* Hospital or Clinic
* School
* Drugstore
* Park
* Farmers Market or Grocery Store

Although we had set these to be our minimum requirements, we later found that it was possible the venue list we had was incomplete (i.e. no elementary or highschools) and just in general, we were working with pretty limited data so we shortlisted based on which neighbourhoods met at least 3 of our requirements.

Finally, we clustered our shortlisted neighbourhoods in order to find out which were from the same borough and in order for us to see any commonalities in neighbourhoods within each cluster.

Based on our results and discussion, it seems like it would be worthwhile to check out Cluster 1 and Cluster 3 for the following reasons:
* Compared to the other three clusters, these two might be quieter in the sense that neither have a railway station in their top 10 most common venues.
* Cluster 1 neighbourhoods have a *lot* of food options and a *lot* of coffee, which parents of young children (and actually anyone, come to think of it) might find quite convenient. 
* Cluster 3 has a bit more variety in its top 10 most common venues. It seems to have quite a few pharmacies and supermarkets, which is handy and several parks, which would be great to relax at.

Again, I refer to the Results and Discussion section for some notes and recommendations on how to improve this research. It would be a lot more insightful to also factor in the sort of data listed there.

## Thank you!

Click [here](#top) to return to the Table of Contents.